# Parsing File and Taking Input

In [2]:
import pandas as pd
import fileinput

In [3]:
def sortDictionary(freq:dict,reverse=False):
    return {k: v for k, v in sorted(freq.items(), key=lambda item: item[1],reverse=reverse)}

In [4]:
def makeDFfromDict(freq:dict,columns:list):
    return pd.DataFrame(list(freq.items()),columns=columns)

In [5]:

filename = 'sample.log'
fileContent = []

for line in fileinput.input(files=filename):
    stringAdded = line.split(' ')
    stringAdded.remove('-')
    stringAdded.remove('-')
    stringAdded[1] += ' '+stringAdded[2]
    stringAdded.pop(2)
    fileContent.append(stringAdded)

# fileContent


# 1. Count Requests per IP Address:

In [6]:
freq = {}
for stringAdded in fileContent:
    if stringAdded[0] in freq.keys():
        freq[stringAdded[0]] +=1
    else:
        freq[stringAdded[0]] = 1
freq = sortDictionary(freq,reverse=True)
print(freq)




{'203.0.113.5': 8, '198.51.100.23': 8, '192.168.1.1': 7, '10.0.0.2': 6, '192.168.1.100': 5}


In [7]:
df = pd.DataFrame(list(freq.items()),columns=["IP Address ","Request"])

df


,IP Address,Request
0,203.0.113.5,8
1,198.51.100.23,8
2,192.168.1.1,7
3,10.0.0.2,6
4,192.168.1.100,5


# 2. Identify the Most Frequently Accessed Endpoint:

In [8]:
urlFreq = {}


for req in fileContent:
    url = req[3]
    if url in urlFreq.keys():
        urlFreq[url] +=1
    else:
        urlFreq[url] = 1
urlFreq = sortDictionary(urlFreq,reverse=True)

print("Most Frequently Accessed Endpoint:")
print(list(urlFreq.keys())[0],"Accessed",urlFreq[list(urlFreq.keys())[0]],"times")


Most Frequently Accessed Endpoint:
/login Accessed 13 times


# 3. Detect Suspicious Activity:

In [28]:
def getSuspiciousActivity(fileContent:list,threshold=10):
    ipFreq = {}

    for req in fileContent:
        
        if req[5] == "401":
            if req[0] in ipFreq.keys():
                ipFreq[req[0]] +=1 
            else:
                ipFreq[req[0]] =1 
    newIpFreq = {}
    for i in ipFreq.keys():
        if ipFreq[i] > threshold:
            newIpFreq[i] = ipFreq[i]
    
    
    print("Suspicious Activity Detected:")
    print("IP Address","\t", "Failed Login Attempts")
    for i in newIpFreq.keys():
        print(i,"\t",newIpFreq[i])
    


getSuspiciousActivity(fileContent,threshold=5)

Suspicious Activity Detected:
IP Address 	 Failed Login Attempts
203.0.113.5 	 8
